In [13]:
import pandas as pd

# Read the CSV file back into a pandas DataFrame
combined_df = pd.read_csv('./combined_df.csv')

In [14]:
# agent.py
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] 

from langchain import OpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

import pandas as pd

In [15]:
combined_df

,DateTime,NI Generation,NI Demand,NI Wind Availability,NI Wind Generation,IE Generation,IE Demand,IE Wind Availability,IE Wind Generation,SNSP,NI Solar Availability,NI Solar Generation,Moyle I/C,IE Hydro,EWIC I/C,Inter-Jurisdictional Flow
0,2014-01-01 00:00:00,637.980,859.360,367.580,365.570,2752.48,2898.72,1053.11,1020.23,0.4579,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-01-01 00:15:00,625.680,855.460,345.940,352.820,2733.59,2868.97,1021.59,995.07,0.4505,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-01-01 00:30:00,614.720,840.000,333.220,339.600,2686.17,2826.42,972.06,932.71,0.4383,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-01-01 00:45:00,588.730,824.250,307.440,313.660,2657.56,2786.94,985.81,959.06,0.4447,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-01-01 01:00:00,593.060,818.840,306.870,315.170,2584.65,2723.94,958.63,920.82,0.4429,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356347,2024-02-29 22:45:00,863.384,809.607,358.616,361.516,3056.91,4010.83,1398.84,1398.63,0.5362,0.0,0.0,349.681,117.28,529.97,-393.59
356348,2024-02-29 23:00:00,843.681,777.295,352.949,355.647,3083.95,4039.40,1562.17,1566.74,0.5681,0.0,0.0,347.268,118.12,529.98,-395.33
356349,2024-02-29 23:15:00,801.090,752.084,314.562,313.371,3147.61,4017.52,1734.05,1736.02,0.5883,0.0,0.0,311.238,118.54,529.98,-342.03
356350,2024-02-29 23:30:00,832.942,727.625,302.065,307.886,3114.66,3933.57,1649.09,1649.66,0.5705,0.0,0.0,320.160,120.76,529.42,-404.84


In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_pandas_dataframe_agent(llm, combined_df, verbose= True)

In [18]:
agent.run("can you plot average wind generation in Ireland in 2023 per month?")



> Entering new AgentExecutor chain...
Thought: To plot the average wind generation in Ireland in 2023 per month, we need to extract the year and month from the DateTime column and then calculate the average wind generation for each month.
Action: python_repl_ast
Action Input: df['DateTime'] = pd.to_datetime(df['DateTime'])We need to convert the 'DateTime' column to a datetime format before extracting the year and month.
Action: python_repl_ast
Action Input: df['DateTime'] = pd.to_datetime(df['DateTime'])Now that the 'DateTime' column is in datetime format, we can proceed to extract the year and month.
Action: python_repl_ast
Action Input: df['Year'] = df['DateTime'].dt.yearNow that we have extracted the year, let's extract the month as well.
Action: python_repl_ast
Action Input: df['Month'] = df['DateTime'].dt.monthNow that we have extracted the year and month, we can calculate the average wind generation for each month in 2023.
Action: python_repl_ast
Action Input: df_2023 = df[df['

'The average wind generation in Ireland in 2023 per month is as follows:\n- January: 1591.65\n- February: 1615.69\n- March: 1489.83\n- April: 1278.94\n- May: 749.41\n- June: 732.02\n- July: 1148.54\n- August: 1165.22\n- September: 1163.52\n- October: 1096.80\n- November: 1549.91\n- December: 2047.16'

In [19]:
def query_agent(agent, query):
    """
    Query an agent and return the response as a string.

    Args:
        agent: The agent to query.
        query: The query to ask the agent.

    Returns:
        The response from the agent as a string.
    """

    prompt = (
        """
            For the following query, if it requires drawing a table, reply as follows:
            {"table": {"columns": ["column1", "column2", ...], "data": [[value1, value2, ...], [value1, value2, ...], ...]}}

            If the query requires creating a bar chart, reply as follows:
            { "response": {your response here},
                "bar": {"columns": ["A", "B", "C", ...], "data": [25, 24, 10, ...]}}

            If the query requires creating a line chart, reply as follows:
            {"response": {your response here},
            "line": {"columns": ["A", "B", "C", ...], "data": [25, 24, 10, ...]}}

            There can only be two types of chart, "bar" and "line".

            If it is just asking a question that does not require a plot, reply as follows:
            {"answer": "answer"}
            Example:
            {"answer": "The title with the highest rating is 'Gilead'"}

            If you do not know the answer, reply as follows:
            {"answer": "I am sorry, I do not know. Maybe rephrase your question can help me."}

            Return all output as a string.

            All strings in "columns" list and data list, should be in double quotes,

            For example: {"columns": ["title", "ratings_count"], "data": [["Gilead", 361], ["Spider's Web", 5164]]}

            Lets think step by step.

            Below is the query.
            Query: 
            """
        + query
    )

    # Run the prompt through the agent.
    response = agent.run(prompt)

    # Convert the response to a string.
    return response.__str__()

In [20]:
# import streamlit as st
import pandas as pd
import json

# from agent import query_agent, create_agent


def decode_response(response: str) -> dict:
    """This function converts the string response from the model to a dictionary object.

    Args:
        response (str): response from the model

    Returns:
        dict: dictionary with response data
    """
    return json.loads(response)

In [21]:
# Create an agent from the CSV file.
# agent = create_agent(df)

# Query the agent.
response = query_agent(agent=agent, query='what is average of wind generation in each month in 2022?')

# Decode the response.
decoded_response = decode_response(response)





> Entering new AgentExecutor chain...
Thought: We need to calculate the average wind generation for each month in the year 2022.
Action: Use groupby to group the data by month and year, then calculate the mean of wind generation.
Action Input: df['DateTime'] = pd.to_datetime(df['DateTime']) to ensure DateTime column is in datetime format
Action Input: df['Month'] = df['DateTime'].dt.month to extract the month from DateTime
Action Input: df['Year'] = df['DateTime'].dt.year to extract the year from DateTime
Action Input: df_2022 = df[df['Year'] == 2022] to filter the data for the year 2022
Action Input: df_2022.groupby(['Month'])['NI Wind Generation'].mean() to calculate the average wind generation for each month in 2022Use groupby to group the data by month and year, then calculate the mean of wind generation. is not a valid tool, try one of [python_repl_ast].I should use the python_repl_ast tool to execute the actions mentioned earlier.

Action: python_repl_ast
Action Input: df['Date

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [22]:
response

'The average wind generation in each month in 2022 is as follows:\n- January: 380.33\n- February: 584.87\n- March: 336.91\n- April: 288.46\n- May: 308.50\n- June: 258.44\n- July: 173.69\n- August: 163.52\n- September: 206.12\n- October: 401.70\n- November: 427.66\n- December: 300.08'

In [23]:
combined_df.columns

Index(['DateTime', 'NI Generation', 'NI Demand', 'NI Wind Availability',
       'NI Wind Generation', 'IE Generation', 'IE Demand',
       'IE Wind Availability', 'IE Wind Generation', 'SNSP',
       'NI Solar Availability', 'NI Solar Generation', 'Moyle I/C', 'IE Hydro',
       'EWIC I/C', 'Inter-Jurisdictional Flow', 'Year', 'Month'],
      dtype='object')

In [30]:
examples = [
    {
        "input": "List the total electricity generation for the island of Ireland on 2023-03-15.",
        "operation": "combined_df[combined_df['DateTime'].str.startswith('2023-03-15')][['IE Generation', 'NI Generation']].sum(axis=1).sum()",
        "description": "This calculates the sum of both 'IE Generation' and 'NI Generation' for March 15, 2023."
    },
    {
        "input": "Find the average wind availability for Northern Ireland in January 2023.",
        "operation": "combined_df[(combined_df['DateTime'].str.startswith('2023-01')) & (combined_df['Year'] == 2023)]['NI Wind Availability'].mean()",
        "description": "This computes the average 'NI Wind Availability' for January 2023."
    },
    {
        "input": "What is the wind curtailment for the Republic of Ireland on 2023-02-10?",
        "operation": "combined_df[combined_df['DateTime'] == '2023-02-10'][['IE Wind Availability', 'IE Wind Generation']].apply(lambda x: x['IE Wind Availability'] - x['IE Wind Generation'], axis=1)",
        "description": "This calculates the wind curtailment ('IE Wind Availability' - 'IE Wind Generation') for February 10, 2023."
    },
    {
        "input": "Calculate the solar curtailment for Northern Ireland on 2023-04-05.",
        "operation": "combined_df[combined_df['DateTime'] == '2023-04-05'][['NI Solar Availability', 'NI Solar Generation']].apply(lambda x: x['NI Solar Availability'] - x['NI Solar Generation'], axis=1)",
        "description": "This calculates the solar curtailment ('NI Solar Availability' - 'NI Solar Generation') for April 5, 2023."
    },
    {
        "input": "What was the wind penetration for the island of Ireland on 2023-01-20?",
        "operation": "combined_df[combined_df['DateTime'] == '2023-01-20'].apply(lambda x: (x['IE Wind Generation'] + x['NI Wind Generation']) / (x['IE Demand'] + x['NI Demand']), axis=1)",
        "description": "This computes the wind penetration for January 20, 2023."
    },
]




In [31]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=5,
    input_keys=["input"],
)

In [80]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a pandas DataFrame. Given an input question, analyze the data and return the answer. Your response should follow specific formats depending on the type of query:

1. If the query can be answered directly without requiring a plot, format your response as a simple answer in a dictionary:
   {"answer": ["Your direct answer here."]}

2. If the query requires creating a bar chart to effectively convey the information, format your response to include both a summary response and the data necessary for plotting as:
   
       {"answer": ["Your summary response here, explaining the key insights or findings.",]
       "bar": ["columns": ["Column1", "Column2", ...], "data": [value1, value2, ...]]}

3. If the query requires creating a line chart, format your response as follows:
   
       {"answer": ["Your summary response here, explaining the key insights or findings."],
        "line": ["columns": ["Column1", "Column2", ...], "data": [value1, value2, ...]]}

4. If you do not know the answer, or if the query is not applicable to the available data, respond with:
   {"answer": ["I am sorry, I do not know. Maybe rephrasing your question can help me."]}

   You MUST double check your command before executing it. If you get an error while executing a query, rewrite the command and try again.

Ensure all strings in the "columns" list and in the data list are enclosed in double quotes. Always think through the steps necessary to derive the answer or prepare the data for plotting, and present the information clearly and concisely. Your primary goal is to provide insights in an accessible format, whether through direct answers or visualizations.
Here are some examples of user inputs and their corresponding SQL queries:"
:"""



few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nPython code: {operation}\n description: {description}"
    ),
    input_variables=["input"],  # Adjust according to actual inputs you're planning to use
    prefix=system_prefix,
    suffix=""
)

# Note: Replace {python_code} with the actual code snippets for interacting with pandas or generating plots.
# Ensure that your examples and the rest of the setup are aligned with the operations your agent will perform.


In [82]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        # MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [83]:
# Example formatted prompt
prompt_val = full_prompt.invoke(
    {
        "input": "How many years are there",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

KeyError: 'Input to ChatPromptTemplate is missing variables {\'"answer"\'}.  Expected: [\'"answer"\', \'input\'] Received: [\'input\', \'agent_scratchpad\']'

In [75]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = create_pandas_dataframe_agent(llm, combined_df, verbose= True)

In [76]:
agent = create_pandas_dataframe_agent(
    llm=llm,
    df=combined_df,
    prompt=full_prompt,
    verbose=True,
    agent_type="openai-tools",
)

/Users/saeed/anaconda3/lib/python3.10/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:242: UserWarning: Received additional kwargs {'prompt': ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=FewShotPromptTemplate(input_variables=[], example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x172108400>, k=5, example_keys=None, input_keys=['input'], vectorstore_kwargs=None), example_prompt=PromptTemplate(input_variables=['description', 'input', 'operation'], template='User input: {input}\nPython code: {operation}\n description: {description}'), suffix='', prefix='You are an agent designed to interact with a pandas DataFrame. Given an input question, analyze the data and return the answer. Your response should follow specific formats depending on the type of query:\n\n1. If the query can be answered directly without requiring a plot, format your response as a s

In [77]:
agent.invoke({"input": "average monthly wind generation in 2023?"})



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Year'] == 2023].groupby('Month')['NI Wind Generation', 'IE Wind Generation'].mean()"}`


ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.
Invoking: `python_repl_ast` with `{'query': "df[df['Year'] == 2023].groupby('Month')[['NI Wind Generation', 'IE Wind Generation']].mean()"}`
responded: It seems there was an error in the query. Let me correct it and try again.

       NI Wind Generation  IE Wind Generation
Month                                        
1              396.682202         1591.651257
2              411.004927         1615.686942
3              262.109727         1489.832678
4              266.453014         1278.941000
5              146.181458          749.411489
6              146.457666          732.021236
7              189.864640         1148.535494
8              235.882139         1165.223807
9              221.029709         1

{'input': 'average monthly wind generation in 2023?',
 'output': 'The average monthly wind generation in 2023 for Northern Ireland (NI) and Ireland (IE) is as follows:\n\n- **Northern Ireland (NI) Wind Generation:**\n  - January: 396.68\n  - February: 411.00\n  - March: 262.11\n  - April: 266.45\n  - May: 146.18\n  - June: 146.46\n  - July: 189.86\n  - August: 235.88\n  - September: 221.03\n  - October: 266.04\n  - November: 261.69\n  - December: 400.06\n\n- **Ireland (IE) Wind Generation:**\n  - January: 1591.65\n  - February: 1615.69\n  - March: 1489.83\n  - April: 1278.94\n  - May: 749.41\n  - June: 732.02\n  - July: 1148.54\n  - August: 1165.22\n  - September: 1163.52\n  - October: 1096.80\n  - November: 1549.91\n  - December: 2047.16'}